In [4]:
import heapq

class Node:
    def __init__(self, name, x, y, node_type='OR'):
        self.name = name
        self.x = x
        self.y = y
        self.node_type = node_type  # OR or AND node
        self.children = []  # Store (child_node, cost) pairs

    def __repr__(self):
        return f"Node({self.name})"

    def __eq__(self, other):
        return self.name == other.name and self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.name, self.x, self.y))

    def add_child(self, child_node, cost):
        """ Adds a child to the node with the corresponding cost """
        self.children.append((child_node, cost))

def heuristic(node1, node2):
    """ Heuristic function: Euclidean distance between two nodes """
    return ((node1.x - node2.x) ** 2 + (node1.y - node2.y) ** 2) ** 0.5

def ao_star(start, goal):
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start))
    solution_graph = {}  # Tracks the parent of each node for backtracking
    cost_dict = {}  # Store cost for each node
    solved = set()  # Nodes already solved

    while open_list:
        f_value, g_value, current_node = heapq.heappop(open_list)

        # If we've reached the goal, reconstruct the path
        if current_node == goal:
            path = []
            while current_node:
                path.append(current_node)
                current_node = solution_graph.get(current_node)
            return path[::-1]  # Reverse the path to start -> goal

        if current_node in solved:
            continue

        if current_node.node_type == 'OR':
            # OR node: select the least-cost child and explore it
            for child, cost in current_node.children:
                new_g_value = g_value + cost
                heapq.heappush(open_list, (new_g_value + heuristic(child, goal), new_g_value, child))
                solution_graph[child] = current_node  # Track the parent
            solved.add(current_node)

        elif current_node.node_type == 'AND':
            # AND node: all children must be expanded
            all_children_solved = True
            for child, cost in current_node.children:
                if child not in solved:
                    all_children_solved = False
                    new_g_value = g_value + cost
                    heapq.heappush(open_list, (new_g_value + heuristic(child, goal), new_g_value, child))
                    solution_graph[child] = current_node  # Track the parent

            if all_children_solved:
                # If all children are solved, mark the AND node as solved
                solved.add(current_node)
                solution_graph[current_node] = solution_graph.get(current_node)

    return None  # If no solution is found

# Define nodes (x, y coordinates, and node type)
nodes = {
    'A': Node('A', 0, 0, node_type='OR'),
    'B': Node('B', 2, 2, node_type='AND'),
    'C': Node('C', 4, 0, node_type='OR'),
    'D': Node('D', 6, 2, node_type='AND'),
    'E': Node('E', 8, 0, node_type='OR'),
}

# Add children (node connections)
nodes['A'].add_child(nodes['B'], 2.5)
nodes['A'].add_child(nodes['C'], 4.0)
nodes['B'].add_child(nodes['D'], 3.5)
nodes['C'].add_child(nodes['D'], 3.0)
nodes['D'].add_child(nodes['E'], 4.5)

# Run AO* algorithm
start_node = nodes['A']
goal_node = nodes['E']
path = ao_star(start_node, goal_node)

if path:
    print("Path found:", " -> ".join(node.name for node in path))
else:
    print("No path found.")


Path found: A -> B -> D -> E
